In [1]:
# Reference : 
# https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/

import sys
import os

# os.environ['THEANO_FLAGS'] = "device=cuda*"
os.environ['KERAS_BACKEND']='tensorflow'
# os.environ['MKL_THREADING_LAYER']='GNU'

import numpy as np
import pandas as pd
from collections import defaultdict
import re

In [2]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
# Merge
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
# from keras import initializations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import constraints

C:\Users\Anneke\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [4]:
import pickle

def save_pickle(path, X):
    with open(path, 'wb') as f:
        pickle.dump(X, f)

def open_pickle(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train = open_pickle("../../data/imdb/imdb_original_preprocessed_xtrain.pickle")
X_test = open_pickle("../../data/imdb/imdb_original_preprocessed_xtest.pickle")
y_train = open_pickle("../../data/imdb/imdb_original_preprocessed_ytrain.pickle")
y_test = open_pickle("../../data/imdb/imdb_original_preprocessed_ytest.pickle")

In [5]:
from nltk import tokenize
from textblob import TextBlob

In [6]:
reviews = [] #sentences
for i in range(len(X_train)):
    sentences = TextBlob(X_train[i]).raw_sentences
    reviews.append(sentences)

In [38]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_train)


C:\Users\Anneke\Anaconda3\lib\site-packages\keras_preprocessing\text.py:174: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [8]:
data = np.zeros((len(X_train), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [9]:
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j<MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k=0
            for _, word in enumerate(wordTokens):
                if k<MAX_SENT_LENGTH and tokenizer.word_index[word]<MAX_NB_WORDS:
                    data[i,j,k] = tokenizer.word_index[word]
                    k=k+1


In [10]:
word_index = tokenizer.word_index

In [11]:
labels = to_categorical(np.asarray(y_train))
labels = np.asarray(y_train)

In [12]:
print('Total %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Total 85439 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000,)


In [13]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [14]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [15]:
print('Number of positive and negative reviews in training and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in training and validation set
10016
2484


In [16]:
GLOVE_DIR = "../../data/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), 'rb')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [17]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [18]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
embedding_layer = Embedding(len(word_index)+1,
                               EMBEDDING_DIM,
                               weights=[embedding_matrix],
                               input_length=MAX_SENT_LENGTH,
                               trainable=True)

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

In [19]:
review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)

In [20]:
preds = Dense(1, activation='sigmoid')(l_lstm_sent)
model = Model(review_input, preds)

In [21]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [22]:
print("model fitting - Hierarchical LSTM")
print(model.summary())

model fitting - Hierarchical LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 200)           8704800   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 8,945,801
Trainable params: 8,945,801
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttLayer, self).__init__(**kwargs)
            
    def build(self, input_shape):
        assert len(input_shape)==3
        self.W = self.init((input_shape[-1],))
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)
        
    def call(self, x, mask=None):
        # u_{it}
        eij = K.tanh(K.dot(x, self.W))

        # alpha
        ai = K.exp(eij)
        weights = ai/K.sum(ai,axis=1).dimshuffle(0,'x')
        
        # s_i
        weighted_input = x * weights.dimshuffle(0,1, 'x')
        return weighted_input.sum(axis=1)
    
    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [24]:
# https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2
# other reference : 
# https://gist.github.com/cbaziotis/6428df359af27d58078ca5ed9792bd6d

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
class AttentionWithContext(Layer):
    def __init__(self,
        W_regularizer=None, u_regularizer=None, b_regularizer=None,
        W_constraint=None, u_constraint=None, b_constraint=None,
        bias=True, **kwargs):
            
        self.supports_masking = False
        self.init = initializers.get('normal')
            
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
            
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
    
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                initializer=self.init,
                                name='{}_W'.format(self.name),
                                regularizer=self.W_regularizer,
                                constraint=self.W_constraint)
            
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                    initializer='zero',
                                    name='{}_b'.format(self.name),
                                    regularizer=self.b_regularizer,
                                    constraint=self.b_constraint)
            
        self.u = self.add_weight((input_shape[-1],),
                                initializer=self.init,
                                name='{}_u'.format(self.name),
                                regularizer=self.u_regularizer,
                                constraint=self.u_constraint)
    
        super(AttentionWithContext, self).build(input_shape)
    
    def compute_mask(self, input, input_mask=None):
        return None
        
    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
            
        if self.bias:
            uit += self.b
                
        uit = K.tanh(uit)
#         ait = K.dot(uit, self.u) # only works on  
        
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
            
        a = K.expand_dims(a)
        weighted_input = x * a
            
        return K.sum(weighted_input,axis=1)
        
    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [25]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)

l_gru = Bidirectional(GRU(50, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(10))(l_gru)
l_att = AttentionWithContext()(l_dense)

sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)

l_gru_sent = Bidirectional(GRU(50, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(10))(l_gru_sent)
l_att_sent = AttentionWithContext()(l_dense_sent)

preds = Dense(1, activation='sigmoid')(l_att_sent)
model = Model(review_input, preds)

In [26]:
model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['acc'])


In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 15, 10)            8590430   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 15, 100)           18300     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 15, 10)            1010      
_________________________________________________________________
attention_with_context_2 (At (None, 10)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 8,609,871
Trainable params: 8,609,871
Non-trainable params: 0
_________________________________________________________________


In [28]:
print('model fitting - Hierarchical attention network')
model.fit(x_train, y_train, validation_data=(x_val, y_val),
           epochs=1, batch_size=50, verbose=1)

model fitting - Hierarchical attention network
Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 258s 13ms/step - loss: 0.6930 - acc: 0.5169 - val_loss: 0.6562 - val_acc: 0.6596


### Relevance sentence Model

In [61]:
# Preprocess
path = r"C:\Users\Anneke\Documents\GitHub\data\imdb-sentence"
X_train_sent = open_pickle(path + r"\imdb_sentence_xtrain.pickle")
X_test_sent = open_pickle(path + r"\imdb_sentence_xtest.pickle")
y_train_sent = open_pickle(path + r"\imdb_sentence_ytrain.pickle")
y_test_sent = open_pickle(path + r"\imdb_sentence_ytest.pickle")

In [62]:
MAX_SEQUENCE = 15

In [63]:
data = np.zeros((len(X_train_sent), MAX_SEQUENCE), dtype='int32')

In [64]:
for i, doc in enumerate(X_train_sent):
    wordTokens = text_to_word_sequence(doc)
    for j, word in enumerate(wordTokens):
        try:
            if j<MAX_SEQUENCE and tokenizer.word_index[word]<MAX_NB_WORDS:
                data[i,j] = tokenizer.word_index[word]
        except KeyError as error:
            continue

In [65]:
data.shape

(1333, 15)

In [66]:
data[0]

array([ 69, 774,   1,   0,   7,   1, 658,   0,  20, 169,  20, 205,   0,
         0,   0])

In [67]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = y_train_sent[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [68]:
x_train = data[:-nb_validation_samples]
y_train = y[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = y[-nb_validation_samples:]

In [127]:
embedding_layer = Embedding(len(word_index)+1,
                               EMBEDDING_DIM,
                               weights=[embedding_matrix],
                               input_length=MAX_SEQUENCE,
                               trainable=True)

In [128]:
sentence_input = Input(shape=(MAX_SEQUENCE,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
sentence_gru = Bidirectional(GRU(50, return_sequences=True))(embedded_sequences)
sentence_dense = TimeDistributed(Dense(10))(sentence_gru)
sentence_att = AttentionWithContext()(sentence_dense)

output = Dense(1, activation='sigmoid')(sentence_att)
model = Model(sentence_input, output)

In [129]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 100)           8544000   
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 15, 100)           45300     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 15, 10)            1010      
_________________________________________________________________
attention_with_context_13 (A (None, 10)                120       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 11        
Total params: 8,590,441
Trainable params: 8,590,441
Non-trainable params: 0
_________________________________________________________________


In [130]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [131]:
print('model fitting - Hierarchical attention network')
model.fit(x_train, y_train, validation_data=(x_val, y_val),
           epochs=10, batch_size=32, verbose=1)

model fitting - Hierarchical attention network
Train on 1067 samples, validate on 266 samples
Epoch 1/10
1067/1067 [==============================] - 8s 8ms/step - loss: 0.7046 - acc: 0.5108 - val_loss: 0.7086 - val_acc: 0.5075
Epoch 2/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.6707 - acc: 0.5829 - val_loss: 0.6656 - val_acc: 0.5714
Epoch 3/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.6398 - acc: 0.6467 - val_loss: 0.7098 - val_acc: 0.5301
Epoch 4/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.5459 - acc: 0.7226 - val_loss: 0.5791 - val_acc: 0.7143
Epoch 5/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.4418 - acc: 0.7957 - val_loss: 0.5098 - val_acc: 0.7594
Epoch 6/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.3571 - acc: 0.8369 - val_loss: 0.6077 - val_acc: 0.7180
Epoch 7/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.3079 - acc: 0.8632 - val

In [132]:
sentence_input = Input(shape=(MAX_SEQUENCE,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
sentence_gru = Bidirectional(GRU(50, return_sequences=False))(embedded_sequences)
# sentence_dense = TimeDistributed(Dense(10))(sentence_gru)
# sentence_att = AttentionWithContext()(sentence_dense)
sentence_dense = Dense(10)(sentence_gru)

output = Dense(1, activation='sigmoid')(sentence_dense)
baseline = Model(sentence_input, output)

In [133]:
baseline.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 100)           8544000   
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 100)               45300     
_________________________________________________________________
dense_29 (Dense)             (None, 10)                1010      
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 8,590,321
Trainable params: 8,590,321
Non-trainable params: 0
_________________________________________________________________


In [134]:
baseline.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [135]:
baseline.fit(x_train, y_train, validation_data=(x_val, y_val),
           epochs=10, batch_size=32, verbose=1)

Train on 1067 samples, validate on 266 samples
Epoch 1/10
1067/1067 [==============================] - 8s 8ms/step - loss: 0.6520 - acc: 0.6298 - val_loss: 0.6261 - val_acc: 0.6805
Epoch 2/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.5206 - acc: 0.7516 - val_loss: 0.5355 - val_acc: 0.7744
Epoch 3/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.3189 - acc: 0.8641 - val_loss: 0.5529 - val_acc: 0.7632
Epoch 4/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.2181 - acc: 0.9100 - val_loss: 0.6814 - val_acc: 0.7444
Epoch 5/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.1970 - acc: 0.9175 - val_loss: 0.5813 - val_acc: 0.7632
Epoch 6/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.1348 - acc: 0.9447 - val_loss: 0.5774 - val_acc: 0.7895
Epoch 7/10
1067/1067 [==============================] - 3s 3ms/step - loss: 0.1128 - acc: 0.9606 - val_loss: 0.8028 - val_acc: 0.7594
Epoch 8/10
1067

In [120]:
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[4].output])
layer_output = get_3rd_layer_output([x_train])[0]

In [119]:
layer_output[0]

array([[-1.2915663, -2.1069176,  0.7979975,  2.8814669,  1.3588922,
        -0.9618858,  1.2557839,  1.4890926, -1.9621711, -2.287041 ],
       [-1.5865849, -2.4945245,  1.2849963,  3.6057577,  1.2882073,
        -1.2771791,  1.4501113,  1.692404 , -2.1654713, -2.8964055],
       [-2.504399 , -3.0841897,  1.9005077,  4.265061 ,  1.6061008,
        -1.8350469,  1.984015 ,  2.227129 , -2.66789  , -3.569856 ],
       [-2.4701836, -2.8196394,  2.0433068,  4.056266 ,  1.5079509,
        -2.2586787,  2.0359666,  2.5391371, -2.440862 , -3.9819386],
       [-2.313811 , -3.0574312,  2.140463 ,  4.497801 ,  1.1588722,
        -2.387347 ,  2.2511702,  2.556943 , -2.7098336, -4.130367 ],
       [-2.6354942, -3.1415088,  2.089699 ,  4.261998 ,  1.3999277,
        -2.5489163,  2.2000186,  2.6368563, -2.5423408, -4.3037806],
       [-2.383254 , -3.4035547,  2.0830734,  4.6236525,  1.1320255,
        -2.560363 ,  2.355486 ,  2.5821612, -2.868234 , -4.3121867],
       [-2.4933615, -3.5699055,  1.924191

In [121]:
layer_output[0]

array([-2.454565 , -2.9458292,  1.9136147,  4.2377877,  1.3164525,
       -2.2750907,  2.0845551,  2.2735617, -2.377486 , -4.0032773],
      dtype=float32)